In [1]:
import pandas as pd
import zipfile
import numpy as np

# READ DATA

## unique_tracks

In [35]:
df_unique = pd.read_csv("data/p02_unique_tracks.txt", sep = "<SEP>", header = None, engine='python')
df_unique.columns = ["track_id", "song_id", "artist", "title"]
print(df_unique)

df_unique.to_pickle("data/cache/unique.pkl")

                  track_id             song_id            artist  \
0       TRMMMYQ128F932D901  SOQMMHC12AB0180CB8  Faster Pussy cat   
1       TRMMMKD128F425225D  SOVFVAK12A8C1350D9  Karkkiautomaatti   
2       TRMMMRX128F93187D9  SOGTUKN12AB017F4F1    Hudson Mohawke   
3       TRMMMCH128F425532C  SOBNYVR12A8C13558C       Yerba Brava   
4       TRMMMWA128F426B589  SOHSBXH12A8C13B0DF        Der Mystic   
...                    ...                 ...               ...   
999995  TRYYYUS12903CD2DF0  SOTXAME12AB018F136      Kiko Navarro   
999996  TRYYYJO128F426DA37  SOXQYIQ12A8C137FBB     Kuldeep Manak   
999997  TRYYYMG128F4260ECA  SOHODZI12A8C137BB3    Gabriel Le Mar   
999998  TRYYYDJ128F9310A21  SOLXGOR12A81C21EB7             Elude   
999999  TRYYYVU12903CD01E3  SOWXJXQ12AB0189F43             Texta   

                                      title  
0                              Silent Night  
1                               Tanssi vaan  
2                         No One Could Ever  

## train_triplets

In [36]:
with zipfile.ZipFile("data/p02_train_triplets.txt.zip", 'r') as zip_ref:
    zip_ref.extractall("data/cache/")
df_plcount = pd.read_csv("data/cache/train_triplets.txt", sep = "\t", header = None, engine='python')
df_plcount.columns = ["user_id", "song_id", "play_count"]
print(df_plcount)
df_plcount.to_pickle("data/cache/play_count.pkl")

                                           user_id             song_id  \
0         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995   
1         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9   
2         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B   
3         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22   
4         b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494   
...                                            ...                 ...   
48373581  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOUHHHH12AF729E4AF   
48373582  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOUJVIT12A8C1451C1   
48373583  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOUSMXX12AB0185C24   
48373584  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOWYSKH12AF72A303A   
48373585  b7815dbb206eb2831ce0fe040d0aa537e2e800f7  SOYYFLV12A58A7A88F   

          play_count  
0                  1  
1                  1  
2                  2  
3                  

## msd_tagtraum_cd2.cls

In [37]:
with open("data/p02_msd_tagtraum_cd2.cls", "r") as tagtraum_fd:
    content = list()
    lines = tagtraum_fd.readlines()
    useful_text = list(filter(lambda row: row[0] != '#', lines))
    for i, line in enumerate(useful_text):
        words = line[:-1].split("\t")
        if len(words) == 2:
            words.append("-")
        content.append(words)

df_gnr = pd.DataFrame(content, columns = ["track_id", "major_gnr", "minor_gnr"])
print(df_gnr)

df_gnr.to_pickle("data/cache/tagtraum.pkl")

                  track_id   major_gnr minor_gnr
0       TRAAAAK128F9318786        Rock         -
1       TRAAAAW128F429D538         Rap         -
2       TRAAABD128F429CF47        Rock       RnB
3       TRAAADJ128F4287B47        Rock         -
4       TRAAADZ128F9348C2E       Latin         -
...                    ...         ...       ...
280826  TRZZZRJ128F42819AF        Rock         -
280827  TRZZZUK128F92E3C60        Folk         -
280828  TRZZZYV128F92E996D     New Age       RnB
280829  TRZZZZD128F4236844        Rock         -
280830  TRZZZZZ12903D05E3A  Electronic         -

[280831 rows x 3 columns]


## mxm_dataset_train

In [38]:
from nltk.stem import PorterStemmer

with zipfile.ZipFile('data/p02_mxm_dataset_train.txt.zip', 'r') as zip_ref:
    zip_ref.extractall("data/cache/")
    
    track_id_list = list()
    mxm_track_id_list = list()
    top_words_list = list()
    song_words_list = list()
    with open("data/cache/mxm_dataset_train.txt", "r") as mxm_fd:
        lines = mxm_fd.readlines()
        useful_text = list(filter(lambda row: row[0] != '#', lines))
        words_line = "".join(map(lambda x: x[1:-1], filter(lambda row: row[0] == '%', useful_text)))
        top_words_list = words_line.split(',')
        
        #apply stemming for words
        ps = PorterStemmer()
        top_words_list = list(map(lambda x: ps.stem(x), top_words_list))

        songs = list(filter(lambda row: row[0]!='%', useful_text))
        for i, song in enumerate(songs):
            clean_song = song[:-1]
            comma1 = clean_song.find(',')
            comma2 = clean_song.find(',', comma1 + 1)       
            res = '{' + clean_song[comma2+1:] + '}'
            raw = eval(res)
            song_words_list.append(raw)
            track_id_list.append(clean_song[:comma1])
            mxm_track_id_list.append(clean_song[comma1+1: comma2])
            

    x = [] # list that will hold data for Data Frame
    num_words = len(top_words_list)
    for d in song_words_list:
        # initializing numpy vector
        temp = np.zeros(num_words, dtype=np.float64)
        for key, val in d.items():
            key -= 1
            temp[key] = val
        x.append(temp) 
    df_words = pd.DataFrame(np.array(x), columns = top_words_list)
    df_words['track_id'] = track_id_list
    df_words['mxm_track_id'] = mxm_track_id_list

    #clean dupl
print(df_words)
df_words.to_pickle("data/cache/mxm.pkl")

           i   the   you    to   and     a    me   it  not    in  ...  bake  \
0        6.0   4.0   2.0   2.0   5.0   3.0   1.0  1.0  0.0   0.0  ...   0.0   
1       10.0   0.0  17.0   8.0   2.0   2.0   1.0  3.0  2.0   3.0  ...   0.0   
2       28.0  15.0   2.0  12.0  22.0   2.0   2.0  4.0  2.0   1.0  ...   0.0   
3        5.0   4.0   3.0   2.0   1.0  11.0   0.0  0.0  4.0   0.0  ...   0.0   
4        4.0   0.0   0.0   5.0   7.0   2.0   4.0  0.0  1.0   1.0  ...   0.0   
...      ...   ...   ...   ...   ...   ...   ...  ...  ...   ...  ...   ...   
210514   0.0   0.0   0.0   0.0   0.0   1.0   0.0  0.0  0.0   0.0  ...   0.0   
210515   1.0  13.0   6.0   5.0   4.0   6.0   0.0  0.0  4.0   4.0  ...   0.0   
210516  13.0   3.0  17.0   5.0   9.0   0.0   8.0  2.0  1.0   0.0  ...   0.0   
210517  10.0   6.0  20.0   0.0   2.0   0.0  30.0  1.0  6.0   3.0  ...   0.0   
210518   4.0  18.0   0.0   3.0   6.0   9.0   1.0  5.0  1.0  12.0  ...   0.0   

        insist  wel  santo   pe  gee  colleg  kad  

## read parsed data

In [2]:
import pandas as pd
import numpy as np

df_words = pd.read_pickle("data/cache/mxm.pkl")
df_plcount = pd.read_pickle("data/cache/play_count.pkl")
df_gnr = pd.read_pickle("data/cache/tagtraum.pkl")
df_unique = pd.read_pickle("data/cache/unique.pkl")

# 1. Top-250 tracks



It should return a dataframe with the following fields: index number, artist
name, track title, play count. The table should be sorted by the play count
descendingly.

In [6]:
songid_raiting = df_plcount[['play_count', "song_id"]].groupby("song_id").sum()
songid_raiting.reset_index("song_id", inplace = True)
top_songid = songid_raiting.sort_values("play_count", ascending = False)#
#merge with songs titles
unique_cleaned = df_unique.drop_duplicates(['song_id'])
final_top = pd.merge(top_songid, unique_cleaned, how='left', on='song_id')
final_top.sort_values("play_count", ascending = False)
task1 = final_top[['play_count', 'artist', 'title']].iloc[:250]
task1

,play_count,artist,title
0,726885,Dwight Yoakam,You're The One
1,648239,Björk,Undo
2,527893,Kings Of Leon,Revelry
3,425463,Harmonia,Sehr kosmisch
4,389880,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
...,...,...,...
245,35253,Triple Six Mafia,Now I'm High_ Really High
246,35245,The Red Jumpsuit Apparatus,Face Down (Album Version)
247,35191,Linkin Park,New Divide (Album Version)
248,35074,Selena Gomez & The Scene,Naturally


# 2. Top-100 tracks by genre


It should return on a given genre a dataframe with the following fields: index
number, artist name, track title, play count. The table should be sorted by
the play count descendingly. You should only use the major genre to perform
the subtask.

In [7]:
songid_raiting = df_plcount[['play_count', "song_id"]].groupby("song_id").sum()
song_raiting_named = pd.merge(songid_raiting, unique_cleaned, on='song_id', how = 'inner')
song_raiting_named_gnr = pd.merge(song_raiting_named, df_gnr, on='track_id', how = 'inner')
song_raiting_named_gnr = song_raiting_named_gnr[['play_count', 'artist', 'title', 'major_gnr']]
song_raiting_named_gnr = song_raiting_named_gnr.sort_values(['major_gnr','play_count'], ascending=False).groupby('major_gnr').head(100)

uniq_gnr = set(song_raiting_named_gnr['major_gnr'].values)
for gnr in uniq_gnr:
    top_gnr = song_raiting_named_gnr[song_raiting_named_gnr['major_gnr'] == gnr]
    top_gnr.index = range(len(top_gnr))
    print('GENER:', gnr)
    print(top_gnr[['play_count', 'artist', 'title']])

GENER: Country
    play_count         artist                             title
0       726885  Dwight Yoakam                    You're The One
1        68150   Shania Twain                              Nah!
2        30535   Taylor Swift                        Tim McGraw
3        23872     Toby Keith                  American Soldier
4        22962   Taylor Swift     I'm Only Me When I'm With You
..         ...            ...                               ...
95        4785    Josh Turner                       All Over Me
96        4755   Craig Morgan                Redneck Yacht Club
97        4729     Toby Keith                      Whiskey Girl
98        4722   Brad Paisley                   I'm Still A Guy
99        4712    Clay Walker  What's It To You (Album Version)

[100 rows x 3 columns]
GENER: Blues
    play_count                artist                         title
0        37942  Sonny Boy Williamson        Don't Start Me Talkin'
1        32888            Tab Benoit          

# 3. Collections: 50 songs about love, 50 songs about war, 50 songs about happiness, 50 songs about loneliness, 50 songs about money.


It should return on a given keyword (love, war, happiness) a dataframe (50
tracks) with the following fields: index number, artist name, track title,
play count. The table should be sorted by the play count descendingly. Try
different approaches to these recommendations:

## A. Baseline - when you look for the keyword and the number of its occurrences in a song, filter using some threshold and then sorting it by the play count,

In [42]:
from nltk.stem import PorterStemmer
import collections

top_numb = 50
threshold = 10

words = ['love', 'war', 'happiness', 'loneliness', 'money']
words_stm = ['love', 'war', 'happi', 'lone', 'money']

#remove duplications in columns due to stemmings column names
duplicated = [item for item, count in collections.Counter(df_words[words_stm].columns).items() if count > 1]

for d in duplicated:
    print(d)
    df_words['tmp'] = df_words[d].sum(axis = 1)
    df_words.drop(d , axis = 1, inplace = True)
    df_words.rename(columns = {'tmp' : d}, inplace=True)

for i, w in enumerate(words_stm):
    selected_songs = df_words[(df_words[w] > threshold)][[w] + ['track_id']]
    top_col = pd.merge(song_raiting_named, selected_songs, how = 'inner', on = 'track_id').sort_values('play_count', ascending=False).iloc[:50]
    top_col.index = range(len(top_col))
    print("\n***", words[i].upper(), "***\n")
    print(top_col[['play_count', 'artist', 'title'] + [w]])


lone

*** LOVE ***

    play_count                     artist  \
0       146978               Bill Withers   
1        65966                 John Mayer   
2        45328          Eminem / Dina Rae   
3        40480              Guns N' Roses   
4        23263                       UB40   
5        23151            Black Eyed Peas   
6        22195                Leona Lewis   
7        20584              Modern Lovers   
8        20410                      Train   
9        20226                 The B-52's   
10       20088             Zac Brown Band   
11       19988           Transvision Vamp   
12       16167                Sense Field   
13       15625                 Neil Young   
14       12516          Justin Timberlake   
15       12459          Akon / Snoop Dogg   
16       11923                    OutKast   
17       11724    Huey Lewis And The News   
18       11395                     Metric   
19       10992                     Twista   
20       10738                Eric 

## B. word2vec - when you look not only for the keyword but for several similar tokens as well using word2vec

In [3]:
#find synonyms by word2vec
import nltk
import gensim
from nltk.data import find
from nltk.stem import PorterStemmer
#for stemming
ps = PorterStemmer()
nltk.download('word2vec_sample')

words = ['love', 'war', 'happiness', 'loneliness', 'money']

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

synonyms = {}
all_synonyms = list()
for w in words:
    top = model.most_similar(positive=w, topn = 10)
    top_stem = set(map(lambda x: ps.stem(x[0]), top))
    top_stem = list(set(top_stem).intersection(set(df_words.columns)))
    if len(top_stem) < 1:
        print('There are no this type of songs:', w)
    else:
        all_synonyms += top_stem
        synonyms[w] = top_stem
    
print(synonyms)

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     /Users/olgaborisova/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


{'love': ['affect', 'hate', 'ador', 'passion', 'love', 'cherish'], 'war': ['war'], 'happiness': ['content', 'whole', 'joy', 'glad', 'enjoy', 'thank', 'happi', 'satisfact', 'spiritu'], 'loneliness': ['empti', 'hopeless', 'despair', 'lone', 'alon', 'solitud'], 'money': ['sum', 'dollar', 'money', 'cash']}


In [4]:
import collections
#repeat from baseline but threshold by sum of synonyms words


#remove duplications in columns due to stemmings column names
duplicated = [item for item, count in collections.Counter(df_words[all_synonyms].columns).items() if count > 1]
print(duplicated)
for d in duplicated:
    print(d)
    df_words['tmp'] = df_words[d].sum(axis = 1)
    df_words.drop(d , axis = 1, inplace = True)
    df_words.rename(columns = {'tmp' : d}, inplace=True)

['lone']
lone


In [8]:
threshold = 10
#select collections
for key, item in synonyms.items():
    s = df_words[item].sum(axis = 1)
    selected_songs = df_words[(s > threshold)]['track_id']
    top_col = pd.merge(song_raiting_named, selected_songs, how = 'inner', on = 'track_id').sort_values('play_count', ascending=False).iloc[:50]
    top_col.index = range(len(top_col))
    print("\n***", key.upper(), "***\n")
    print(top_col[['play_count', 'artist', 'title']])


*** LOVE ***

    play_count                     artist  \
0       146978               Bill Withers   
1        74654               Rise Against   
2        65966                 John Mayer   
3        45328          Eminem / Dina Rae   
4        40480              Guns N' Roses   
5        23263                       UB40   
6        23151            Black Eyed Peas   
7        22195                Leona Lewis   
8        20584              Modern Lovers   
9        20410                      Train   
10       20226                 The B-52's   
11       20088             Zac Brown Band   
12       19988           Transvision Vamp   
13       19959           Three Days Grace   
14       16167                Sense Field   
15       15625                 Neil Young   
16       12516          Justin Timberlake   
17       12459          Akon / Snoop Dogg   
18       11923                    OutKast   
19       11724    Huey Lewis And The News   
20       11395                     Metri

## C. classification task -you may label your data and try classification algorithms that will predict for the other part of the dataset if a track belongs to a specific class.

In [4]:
#extracted new database to labele part of songs

import os
import sys
import sqlite3
import pandas as pd

sqliteConnection = sqlite3.connect('data/lastfm_tags.db')

sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor = sqliteConnection.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

df_tags = pd.read_sql_query("SELECT * from tags", sqliteConnection)
df_tids = pd.read_sql_query("SELECT * from tids", sqliteConnection)
df_tid_tag = pd.read_sql_query("SELECT * from tid_tag", sqliteConnection)

[('tags',), ('tids',), ('tid_tag',)]


In [5]:
df_tids['index'] = range(1, df_tids.shape[0] + 1)
df_tags['index'] = range(1, df_tags.shape[0] + 1)

useful_tags = ['love', 'war', 'happ', 'lone ', 'money']

df_tags_useful = pd.DataFrame()
for tag in useful_tags:
    tag_df = df_tags[df_tags['tag'].apply(lambda x: x.find(tag)) > -1]
    tag_df = tag_df.assign(new_tag = tag)
    df_tags_useful = pd.concat([df_tags_useful, tag_df], axis = 0)

df_tags_useful['index'] = range(1, df_tags_useful.shape[0]+1)
df_tags_useful

,tag,index,new_tag
30,lovely,1,love
170,slow soul love jams,2,love
174,elandal loved,3,love
188,loved,4,love
351,love of my life,5,love
...,...,...,...
516307,Beat Like I get money,13410,money
518302,no money down,13411,money
519321,i want money,13412,money
519326,I need money,13413,money


In [6]:
#merge df_tids
full_merge1 = pd.merge(df_tid_tag, df_tids, left_on = 'tid', right_on = 'index', how = 'inner')
#merge df_tid_tag
full_merge2 = pd.merge(full_merge1, df_tags_useful, left_on = 'tag', right_on = "index", how = 'inner')
full_merge2.drop(['tid_x','tag_x', 'val', 'index_x', 'index_y'], axis = 1, inplace = True)
full_merge2.columns = ['track_id', 'tag', 'category_tag']
full_merge2 = full_merge2.groupby("category_tag").head(10000)
full_merge2

,track_id,tag,category_tag
0,TRCCCYE12903CFF0E9,lovely,love
1,TRCCDJG128E0785144,lovely,love
2,TRCCEGW128F92FE7DE,lovely,love
3,TRCCZOW128F146A22D,lovely,love
4,TRCCTGM12903D05323,lovely,love
...,...,...,...
5179976,TRUBRRH12903CC30D6,money friend burro banton,money
5179977,TRUSPHF128F425E07F,money friend burro banton,money
5179978,TRUFAUR12903D01A94,money friend burro banton,money
5179979,TRUFDHB128F425EBC6,money friend burro banton,money


In [7]:
labels = full_merge2[['track_id', 'category_tag']]
labeles_words = pd.merge(df_words, labels, on = 'track_id', how = "left")


In [8]:
labeled_subset = labeles_words[labeles_words['category_tag'].isnull() == False]
unlabeled_subset = labeles_words[labeles_words['category_tag'].isnull()]

In [9]:
selected_columns = labeled_subset.drop(['track_id', 'mxm_track_id', 'category_tag'], axis = 1)
X_train = np.array(selected_columns)
y_train = labeled_subset['category_tag'].values
selected_columns = unlabeled_subset.drop(['track_id', 'mxm_track_id', 'category_tag'], axis = 1)
X_test = np.array(selected_columns)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2, n_jobs=-1)

knn.fit(X_train, y_train)

learned_labels = knn.predict(X_test)
learned_labels

array(['lone ', 'lone ', 'happ', ..., 'happ', 'lone ', 'happ'],
      dtype=object)

In [11]:
unlabeled_subset_learned = unlabeled_subset.assign(category_tag = learned_labels)
full_labeled_mxm = pd.concat([unlabeled_subset_learned, labeled_subset], axis = 0)[['track_id', 'category_tag']]

In [12]:
answer = pd.merge(song_raiting_named, 
        full_labeled_mxm, 
        how = 'inner', 
        on = 'track_id')

cat_ds = answer.sort_values('play_count', ascending=False).groupby('category_tag').head(50)

In [15]:
useful_tags = ['love', 'war', 'happ', 'lone ', 'money']
words = ['love', 'war', 'happiness', 'loneliness', 'money']
for i, tag in enumerate(useful_tags):
        print("\n***", words[i].upper(), "***\n")
        tmp_cat = cat_ds[cat_ds['category_tag'] == tag][['play_count', 'artist', 'title']]
        tmp_cat.index = range(len(tmp_cat))
        print(tmp_cat)


*** LOVE ***

    play_count                   artist  \
0       155717       Lil Wayne / Eminem   
1       111615                Lady GaGa   
2        98667                Metallica   
3        87745               John Mayer   
4        76893                The Verve   
5        65585                   Eagles   
6        64374                 Thursday   
7        64000              Linkin Park   
8        63937                 Owl City   
9        62438          Black Eyed Peas   
10       55921   Florence + The Machine   
11       52834                 N.E.R.D.   
12       50763                   M.I.A.   
13       50155                  Rihanna   
14       49126        Another Sunny Day   
15       47210            Johnny Winter   
16       43640          Vampire Weekend   
17       43280             Jack Johnson   
18       42429          LMFAO / Lil Jon   
19       42348  Five Finger Death Punch   
20       42059           Lynyrd Skynyrd   
21       41493                 Coldplay

# 4. DataPeople similar to you listen: 10 recommendations for each user. Collaborative filtering approach.

look at https://habr.com/ru/post/150399/


For these recommendations, you need to use the train/test split approach. In
this case, the best practice is to cut a sub-matrix from the user-item matrix
for the test dataset and the other parts to use for the train.
To assess your recommendations use the metric p@k (precision at k). It shows
the percentage of the correct recommendations from your list. It means, that
if you gave a user 10 tracks to listen and if they liked 3 of them (they really
listen to them in the test dataset), then the p@k will be equal to 30%. Calculate
the average p@k for your recommendations. It should be at least greater than
10%.
The script should return 10 recommendations for a given user in a dataframe:
index number, artist name, track title. The table should be sorted descendingly
by the “likelihood” that any given user will “like” the track.

In [2]:
import pandas as pd
import numpy as np

df_plcount = pd.read_pickle("data/cache/play_count.pkl")
df_unique = pd.read_pickle("data/cache/unique.pkl")
#remove songs which was listend less than N times in sum
min_listend = 1

plcount1 = df_plcount[['song_id', 'play_count']].groupby('song_id').sum()
print('old_song_numbers:', plcount1.shape[0])
keep_song_index = plcount1[plcount1['play_count'] > min_listend].index
print("new_song_numbers:", len(keep_song_index))
plcount1 = df_plcount[df_plcount['song_id'].isin(keep_song_index)]
del df_plcount, keep_song_index, min_listend
plcount1

old_song_numbers: 384546
new_song_numbers: 362462


,user_id,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
...,...,...,...
48373581,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUHHHH12AF729E4AF,2
48373582,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUJVIT12A8C1451C1,1
48373583,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUSMXX12AB0185C24,1
48373584,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOWYSKH12AF72A303A,3


In [3]:
#split train_test, by songs for any user 
from sklearn.model_selection import train_test_split

train_songs, test_songs, train_users, test_users = train_test_split(plcount1[['song_id', 'play_count']], plcount1['user_id'], random_state = 21, test_size = 0.4)

train = pd.concat([train_songs, train_users], axis = 1)
test = pd.concat([test_songs, test_users], axis = 1)

#use user num if they are in train and test
train = train[train['user_id'].isin(test['user_id'].values)]
test = test[test['user_id'].isin(train['user_id'].values)]
del train_songs, test_songs, train_users, test_users

In [4]:
#numerate songs from 0 to len(song) for matrix creatioin
uniq_songs = plcount1['song_id'].value_counts().index
uniq_songs_numerated = pd.Series(index = uniq_songs, data = range(len(uniq_songs)))
len_usongs = len(uniq_songs_numerated)
#numerate users from 0 to len(users) for matrix creatioin
uniq_users = train['user_id'].value_counts().index
uniq_users_numerated = pd.Series(index = uniq_users, data = range(len(uniq_users)))
len_uusers = len(uniq_users_numerated)

train['song_num'] = train['song_id'].map(uniq_songs_numerated)
train['user_num'] = train['user_id'].map(uniq_users_numerated)

test['song_num'] = test['song_id'].map(uniq_songs_numerated)
test['user_num'] = test['user_id'].map(uniq_users_numerated)

del uniq_songs, uniq_users

In [5]:
# normalization -> all - mean
avarage_byuser = train[['user_num', 'play_count']].groupby('user_num').mean()
avarage_byuser.reset_index('user_num', inplace = True)
tmp = pd.merge(train, avarage_byuser, how = 'left', on='user_num')
tmp['play_count_x'] = tmp['play_count_x'] - tmp['play_count_y']
tmp.rename(columns = {'play_count_x':'play_count'}, inplace = True)
train = tmp
del avarage_byuser, tmp
train

,song_id,play_count,user_id,song_num,user_num,play_count_y
0,SODUHAW12A58A771D0,-0.018868,2c31e0291cf8f85c385a3edfd2ff783b79c65a9f,7087,137502,1.018868
1,SOLPDDR12A8C13A074,-0.600000,51ec8378094eae1df19e3f0aa534772217654592,62222,45493,1.600000
2,SODJWHY12A8C142CCE,1.950000,242e16e3ba9d3e726d7c26e5df3f9ce3ca29e587,8,112549,2.050000
3,SODGJKH12AAA8C9487,-1.560000,363b59ed787ec9d37ec678c220563d3247686d6e,55,343662,2.560000
4,SOUGUJO12A8C13D8A6,-1.564356,bbe751456f045637ffacb4347b66538a35691eac,45254,40143,2.564356
...,...,...,...,...,...,...
29003299,SOGJDAO12AB0182AFA,0.363636,e12c8af008608e31c9f364cd44d678d08cfe9bc4,20362,675413,3.636364
29003300,SOGVXXE12A6D4F7596,-2.377778,40d9a21b25dac20336f1bc008dde7ea6755e7cc3,5252,19511,3.377778
29003301,SOLDIAN12A8C13F496,-1.426829,995d6bcd1aed0b4cad0a7c01cfe5097cfaf3e97d,10021,11433,2.426829
29003302,SOLDUGJ12AC9E156A7,11.202532,44ab84507d3a16769b47909de5cafaa5a0f52eb5,60491,67965,3.797468


In [6]:
#create sparse matrix 
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from scipy.sparse import spdiags

#This method converts a list of (user, item, rating, time) to a sparse matrix
def user_feachers_matrix(user_num, song_num, play_count):
    return csr_matrix((play_count, (user_num, song_num)),
                             shape=(len_uusers, len_usongs))


# Create sparse matrices user_songs
user_song_mtx = user_feachers_matrix(np.array(train['user_num']), np.array(train['song_num']), np.array(train['play_count']))

#normolize matrix for easy cos simularity calculate
user_song_mtx_norm = normalize(user_song_mtx.tocsr()).tocsr()
del user_song_mtx

In [7]:
# for fast calculation
train_series = train.set_index('user_num')
train_series = train_series['song_num']
#sond_num_artist
uniq_songs_numerated.name = 'song_num'
songnum_artist = pd.merge(uniq_songs_numerated, df_unique[['song_id', 'artist']], right_on = 'song_id', left_index = True, how='inner')
songnum_artist = songnum_artist.set_index('song_num')
songnum_artist = songnum_artist['artist']

In [8]:
#now find more N similar users
#precition for one user

import time
from sklearn.metrics.pairwise import cosine_similarity#(X, Y=None, dense_output=True)
from scipy.stats import pearsonr 
from sklearn.metrics.pairwise import cosine_similarity
#colaboration filtration
def track_predict(user_num, sparse_matrix, similar_users_count): 
    #cosin similarity
    user_vector = sparse_matrix[user_num, :]
    cosin = user_vector.dot(sparse_matrix.T)

    #extract N the most similar

    row, col = cosin.nonzero()
    data_cos = cosin.toarray()[0][col]
    similarity_dict = dict(zip(col, data_cos))
    top_sim_users = dict(sorted(similarity_dict.items(), key = lambda x: x[1], reverse = True)[1:similar_users_count + 1])
    
    simils_items = sparse_matrix[list(top_sim_users.keys()), :]
    
    #multiple user_similarity coeff to every matrix row
    # selected_cos = cosin.tocsr()[0, list(top_sim_users.keys())]
    # simils_items = simils_items.astype(np.float32)
    # for i in range(similar_users_count):
    #    simils_items[i, :] = simils_items[i, :] * selected_cos[0, i]
    
    #summary by useres to see top trackes
    final_predicted_raitings = simils_items.sum(axis = 0)
    
    #drop columns which user listend in train
    user, users_listend = sparse_matrix[user_num, :].nonzero()

    row, col = final_predicted_raitings.nonzero()
    new_col = list(set(col) - set(users_listend))
    #extrack song numbers
    data_raiting = np.array(final_predicted_raitings[0, new_col])[0].tolist()
    dict_raiting = dict(zip(list(new_col), data_raiting))
    #sorted_songs = dict(sorted(dict_raiting.items(), key = lambda x: x[1], reverse = True)[:100])
    sorted_songs = dict(sorted(dict_raiting.items(), key = lambda x: x[1], reverse = True)[:10])
    return sorted_songs.keys()

def select_prediction(user_num, dict_coefs, artist_coeff):
    new_dict_coefs = dict_coefs
    songs_num = dict_coefs.keys()
    song_artst_predicted = songnum_artist.loc[songs_num].values
    song_artst_listened = songnum_artist.loc[train_series.loc[user_num]].values
    for i, song in enumerate(songs_num):
        if song_artst_predicted[i] in song_artst_listened:
            new_dict_coefs[song] += artist_coeff
    top = list(dict(sorted(new_dict_coefs.items(), key = lambda x: x[1], reverse = True)).keys())[:10]
    return (top)
    


In [9]:
user_id = 'ec6dfcf19485cb011e0b22637075037aae34cf26'
user_num = uniq_users_numerated.loc[user_id]
songs = track_predict(user_num, user_song_mtx_norm, 5)
#top10_songs_num = select_prediction(user_num, dict_coefs, 1)
#songs_ids = uniq_songs_numerated.loc[uniq_songs_numerated.isin(top10_songs_num)].index.tolist()
songs_ids = uniq_songs_numerated.loc[uniq_songs_numerated.isin(songs)].index.tolist()
songs = df_unique[df_unique.song_id.isin(songs_ids)][['artist', 'title']]
songs.index = range(len(songs))
print(songs)
del user_id, user_num, songs_ids, songs

                   artist                                        title
0  Carbon Based Lifeforms                               Photosynthesis
1  Hercules & Love Affair                                        Blind
2  Carbon Based Lifeforms                                     MOS 6581
3  Carbon Based Lifeforms                            Hydroponic Garden
4    Benny Goodman Sextet                                     Cherokee
5                     Air                                   Mike Mills
6                 Genesis  Looking For Someone (2009 Digital Remaster)
7         Alice In Chains                                Your Decision
8  Amsterdam Klezmer Band                              Alles kan beter
9               Los Twist                                     Invasion


In [10]:
import gc
gc.collect()

0

In [17]:

#calculate metric
train_series = train.set_index('user_num')
train_series = train_series['song_num']
test_series = test.set_index('user_num')
test_series = test_series['song_num']

average = 0
start_time = time.time()
similar_users_count = 10

for n in range(10000):
    test_user = test_series.loc[n]
    if isinstance(test_user, (int, np.integer)):
        test_set = set([test_series.loc[n]])
    elif len(test_user) > 1:
        test_set = set(test_series.loc[n].values)
    else:
        continue
    predicted = set(track_predict(n, user_song_mtx_norm, similar_users_count))
    metric = len(predicted.intersection(test_set)) / 10
    average += metric
    if n % 100 == 0:
        past_time = (time.time() - start_time) / 60
        print("n =", n, "sum = ", average, "past time in min = ", past_time)
        if n % 1000 == 0:
            similar_users_count += 10

average /= n + 1
print(average)

n = 0 sum =  0.1 past time in min =  0.011757481098175048
n = 100 sum =  14.399999999999988 past time in min =  0.7462666670481364
n = 200 sum =  26.199999999999996 past time in min =  1.457024617989858
n = 300 sum =  38.00000000000002 past time in min =  2.1181978662808736
n = 400 sum =  50.200000000000045 past time in min =  2.7523807644844056
n = 500 sum =  62.700000000000095 past time in min =  3.3705015341440836
n = 600 sum =  71.79999999999998 past time in min =  4.0321100155512495
n = 700 sum =  83.29999999999987 past time in min =  4.695018366972605
n = 800 sum =  93.4999999999998 past time in min =  5.333034070332845
n = 900 sum =  100.99999999999969 past time in min =  5.985640418529511
n = 1000 sum =  111.8999999999996 past time in min =  6.638291418552399
n = 1100 sum =  123.79999999999946 past time in min =  7.274241451422373
n = 1200 sum =  134.59999999999937 past time in min =  7.9250236511230465
n = 1300 sum =  145.99999999999926 past time in min =  8.528076283137004
n 

# 5. dataPeople who listen to this track usually listen: 10 recommendations for each track. Collaborative filtering.

The same things applied to these recommendations: use train/test split, use
p@k. If you gave a user 10 tracks to listen and if they liked 3 of them (they
really listen to them in the test dataset), then the p@k will be equal to 30%.
Calculate the average p@k for your recommendations. It should be at least greater
than 10%.
The script should return 10 recommendations for a given track in a dataframe:
index number, artist name, track title. The table should be sorted descendingly
by the “likelihood” that any given user will “like” the track.

https://habr.com/ru/company/ivi/blog/232843/

In [11]:
item_user = user_song_mtx_norm.T
item_user.shape

(362462, 1018633)

In [12]:
from sklearn.preprocessing import normalize
from scipy.sparse import spdiags

cosine_item = item_user.dot(item_user.T)

In [13]:
#zero for diagonal
diag = spdiags(-cosine_item.diagonal(), [0], *cosine_item.shape, format='csr')
cosine_sim_matrix = cosine_item + diag
del cosine_item, diag

In [14]:
from scipy.sparse import vstack
import numpy as np

cosine_sim_matrix = cosine_sim_matrix.tocsr()
m = 10

rows = []
#for row_id in np.unique(cosine_sim_matrix.nonzero()[0]):
for row_id in range(len(uniq_songs_numerated)):
    row = cosine_sim_matrix[row_id]  # исходная строка матрицы
    if row.nnz > m:
        work_row = row.tolil()
        #nullify not in top
        work_row[0, row.nonzero()[1][np.argsort(row.data)[-m:]]] = 0
        row = row - work_row.tocsr()
    rows.append(row)
topk_matrix = vstack(rows) 
topk_matrix = normalize(topk_matrix)
del cosine_sim_matrix


In [15]:
import gc
gc.collect()

0

In [29]:
song_id = 'SOUHHHH12AF729E4AF'
print("SONG: ")
print(df_unique[df_unique.song_id == song_id][['artist', 'title']])
song_num = uniq_songs_numerated.loc[song_id]
row, cols = topk_matrix[song_num, :].nonzero()
songs_ids = uniq_songs_numerated.loc[uniq_songs_numerated.isin(cols)].index
print("SIMILAR")
pr = df_unique[df_unique.song_id.isin(songs_ids)][['artist', 'title']]
pr.index = range(len(pr))
print(pr)


SONG: 
                                                   artist       title
945994  Eminem / Obie Trice / Stat Quo / Bobby Creekwa...  We're Back
SIMILAR
                                              artist                  title
0                                   Eminem / 50 Cent              The Re-Up
1                                             Eminem        Shady Narcotics
2                                         Obie Trice                Cry Now
3                                            Suicide             Be Bop Kid
4                                       Roman Flugel            Geht's Noch
5  Nate Dogg / Eminem / Obie Trice / Bobby Creekw...     Shake That (Remix)
6                                      Junior Murvin       Police & Thieves
7                          Eminem / DMX / Obie Trice            Go To Sleep
8                              Screamin' Jay Hawkins  Heart Attack and Vine
9                                       Dar Williams      Are You Out There


In [24]:
import time
test_series = test.set_index('user_num')
test_series = test_series['song_num']
avarage = 0
start_time = time.time()

for n in range(10000):
    test_user = test_series.loc[n]
    user_vect = user_song_mtx_norm[n, :].T
    if isinstance(test_user, (int, np.integer)):
        test_set = set([test_series.loc[n]])
    elif len(test_user) > 1:
        test_set = set(test_series.loc[n].values)
    else:
        continue
    x = topk_matrix.dot(user_vect).tolil()
    for i, j in zip(*user_vect.nonzero()):
        x[i, j] = 0
    
    x = x.T.tocsr()
    quorum = 10
    data_ids = np.argsort(x.data)[-quorum:][::-1]
    #result = []
    answer = []
    for arg_id in data_ids:
        row_id, p = x.indices[arg_id], x.data[arg_id]
        answer.append(row_id)
        #result.append({"item_num": row_id, "weight": p})
    predicted = set(answer)
    metric = len(predicted.intersection(test_set)) / 10
    avarage += metric
    if n % 100 == 0:
        past_time = (time.time() - start_time) / 60
        print("n =", n, "sum = ", avarage, "past time in min = ", past_time)

avarage /= n + 1
print(avarage)

n = 0 sum =  0.5 past time in min =  0.01095720132191976
n = 100 sum =  18.499999999999993 past time in min =  1.0898713032404581
n = 200 sum =  33.50000000000003 past time in min =  2.1652891993522645
n = 300 sum =  50.500000000000085 past time in min =  3.2408734877904255
n = 400 sum =  64.40000000000012 past time in min =  4.318377888202667
n = 500 sum =  79.30000000000003 past time in min =  5.396473670005799
n = 600 sum =  89.69999999999995 past time in min =  6.47511393626531
n = 700 sum =  105.99999999999987 past time in min =  7.553447250525156
n = 800 sum =  117.99999999999982 past time in min =  8.631449484825135
n = 900 sum =  128.6999999999997 past time in min =  9.735368684927623
n = 1000 sum =  140.59999999999957 past time in min =  10.828480470180512
n = 1100 sum =  154.99999999999952 past time in min =  11.91102305650711
n = 1200 sum =  164.79999999999927 past time in min =  12.995642956097921
n = 1300 sum =  177.1999999999992 past time in min =  14.079694533348084
n = 

# Bonuses:

## a. Top listened by artist

In [1]:
import pandas as pd
import numpy as np

df_unique = pd.read_pickle("data/cache/unique.pkl")
df_plcount = pd.read_pickle("data/cache/play_count.pkl")

In [23]:
import pandas as pd
import numpy as np

user_id = 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'
df_unique = pd.read_pickle("data/cache/unique.pkl")
df_plcount = pd.read_pickle("data/cache/play_count.pkl")

listened_songsid = df_plcount[['song_id', 'play_count']][df_plcount['user_id'] == user_id]
listened_songsid = listened_songsid[['song_id', 'play_count']].groupby('song_id').sum().sort_values('play_count', ascending = False)[:20].index
print('USER LISTENED top 20:')
ls = df_unique[df_unique.song_id.isin(listened_songsid)][['artist', 'title']]
ls.index = range(len(ls))
print(ls)
song_artist_listened = df_unique[df_unique.song_id.isin(listened_songsid)]['artist']
same_artists = df_unique[(df_unique.artist.isin(song_artist_listened)) & ~(df_unique.song_id.isin(listened_songsid))][['artist', 'title', 'song_id']]
top = pd.merge(same_artists, df_plcount[['song_id', 'play_count']], on = 'song_id', how = 'inner')
top = top.groupby(['song_id', 'artist', 'title']).sum().sort_values('play_count', ascending = False)[:10]
top.reset_index(inplace=True)
top.index = range(len(top))
print('\nRECOMENDATION:')
print(top[['artist', 'title']])

USER LISTENED top 20:
                 artist                                              title
0         Paco De Lucia                                    Entre Dos Aguas
1            Josh Rouse                           Carolina (Album Version)
2    Panic At The Disco                   Behind The Sea [Live In Chicago]
3            Puff Daddy  I'll Be Missing You (Featuring Faith Evans & 1...
4            Josh Rouse                          Wonderful (Album Version)
5       Jimmy Eat World                                         The Middle
6          Jack Johnson                                           The Cove
7         Bobby Freeman                                 Do You Wanna Dance
8            Chris Bell                                     Speed Of Sound
9   Héroes del Silencio                       Apuesta Por El Rock 'N' Roll
10        Jorge Drexler                           12 segundos de oscuridad
11        Jorge Drexler                                       High and dry
12 

# b. Genre and tag

In [10]:
import pandas as pd
import numpy as np

df_words = pd.read_pickle("data/cache/mxm.pkl")
df_plcount = pd.read_pickle("data/cache/play_count.pkl")
df_gnr = pd.read_pickle("data/cache/tagtraum.pkl")
df_unique = pd.read_pickle("data/cache/unique.pkl")

In [11]:
genre_list = ['Rock', 'Electronic', 'Pop', 'Jazz', 'Rap', 'RnB', 'Metal', 'Country', 'Reggae', 'Blues', 'Latin', 'Folk', 'Punk', 'World', 'New Age']
tag_list = ['love', 'war', 'happiness', 'loneliness', 'money']

In [12]:
tag = 'happiness'
genre = 'Rock'

if genre not in genre_list:
    print('Wrong genre')
if tag not in tag_list:
    print('Wrong tag')

In [13]:
#select songs with this genre
songid_raiting = df_plcount[['play_count', "song_id"]].groupby("song_id").sum()
unique_cleaned = df_unique.drop_duplicates(['song_id'])
song_raiting_named = pd.merge(songid_raiting, unique_cleaned, on='song_id', how = 'inner')
song_raiting_named_gnr = pd.merge(song_raiting_named, df_gnr, on='track_id', how = 'inner')
song_raiting_named_gnr = song_raiting_named_gnr[['play_count', 'artist', 'title', 'major_gnr']]
song_raiting_named_gnr = song_raiting_named_gnr.sort_values(['major_gnr','play_count'], ascending=False)
top_gnr = song_raiting_named_gnr[song_raiting_named_gnr['major_gnr'] == genre]
top_gnr

,play_count,artist,title,major_gnr
5779,648239,Björk,Undo,Rock
128838,527893,Kings Of Leon,Revelry,Rock
40213,425463,Harmonia,Sehr kosmisch,Rock
96529,292642,OneRepublic,Secrets,Rock
137161,268353,Tub Ring,Invalid,Rock
...,...,...,...,...
172726,1,Nikki & The Corvettes,Let's Go,Rock
172742,1,Dave Mason,Will You Still Love Me Tomorrow?,Rock
172774,1,SINNER,Under The Gun,Rock
172778,1,Reverend Bizarre,Aleister,Rock


In [15]:
#select by tag 
#word2vect
import nltk
import gensim
from nltk.data import find
from nltk.stem import PorterStemmer
ps = PorterStemmer()
nltk.download('word2vec_sample')

#find synonyms
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)
top = model.most_similar(positive=tag, topn = 10)
top_stem = set(map(lambda x: ps.stem(x[0]), top))
top_stem = list(set(top_stem).intersection(set(df_words.columns)))


import collections

#remove duplications in columns due to stemmings column names
duplicated = [item for item, count in collections.Counter(df_words[top_stem].columns).items() if count > 1]
for d in duplicated:
    df_words['tmp'] = df_words[d].sum(axis = 1)
    df_words.drop(d , axis = 1, inplace = True)
    df_words.rename(columns = {'tmp' : d}, inplace=True)

threshold = 15
#select collections
s = df_words[top_stem].sum(axis = 1)
selected_songs = df_words[(s > threshold)]['track_id']
tag_col = pd.merge(song_raiting_named, selected_songs, how = 'inner', on = 'track_id').sort_values('play_count', ascending=False)
tag_col

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     /Users/olgaborisova/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


,song_id,play_count,track_id,artist,title
25,SOHXHWB12A58A7ACD1,7935,TRWWDAU128F92EF6A8,Andrew Gold,Thank You for Being a Friend
0,SOALYAV12A8C133DC2,3468,TRSBCEX128F426EF9D,Ginuwine,Differences
32,SOJZEXX12A6D4F6247,3268,TRYUYQX128F14563AC,Regina Spektor,Chemo Limo (Remastered Version)
21,SOFWNQX12A6D4FCE21,2699,TROCYCX128F423A770,Lupe Fiasco,Go Go Gadget Flow (Explicit Album Version)
2,SOAPXUQ12A8C13232E,2260,TREWJYR128F42652C2,Blood_ Sweat & Tears,You've Made Me So Very Happy
...,...,...,...,...,...
102,SOYFBEO12A8C131DF0,3,TRMFKVH128F423EDF6,Bis,Making People Normal
88,SOUMPYV12AB01862A7,2,TRVBEVK128F9321FF2,Robyn Hitchcock,Happy The Golden Prince
101,SOYBJVB12A6701E9F7,1,TRSYSDO128E078AFD7,Brandy,Happy ( LP Version )
61,SOPBWQQ12AB0188C16,1,TRJDRMR12903CAA80B,Ronnie Dove,Happy Summer Days


In [20]:
#merge genre and tag

final = pd.merge(top_gnr, tag_col, left_on = ['artist', 'title'], right_on = ['artist', 'title'], how = 'inner', suffixes=('_gnr', '_tag'))
final[['artist', 'title']][:10]

,artist,title
0,Andrew Gold,Thank You for Being a Friend
1,Regina Spektor,Chemo Limo (Remastered Version)
2,Blood_ Sweat & Tears,You've Made Me So Very Happy
3,Travis,Happy
4,Good Old War,Just Another Day
5,The Boy Least Likely To,I'm Glad I Hitched My Apple Wagon To Your Star
6,The Sundays,Joy
7,Simon & Garfunkel,You Can Tell The World
8,Bachman-Turner Overdrive,Thank You For The Feelin'
9,The Black Crowes,Make Glad


## C. 250 least popular songs

In [2]:
import pandas as pd
import numpy as np

df_plcount = pd.read_pickle("data/cache/play_count.pkl")
df_unique = pd.read_pickle("data/cache/unique.pkl")

songid_raiting = df_plcount[['play_count', "song_id"]].groupby("song_id").sum()
songid_raiting.reset_index("song_id", inplace = True)
top_songid = songid_raiting.sort_values("play_count")
#merge with songs titles
unique_cleaned = df_unique.drop_duplicates(['song_id'])
final_top = pd.merge(top_songid, unique_cleaned, how='left', on='song_id')
final_top.sort_values("play_count")
task1 = final_top[['play_count', 'artist', 'title']].iloc[:250]
task1

,play_count,artist,title
0,1,Andrew Bird,Ouo
1,1,Krista Detor,Icarus
2,1,Elsa Garcia,No Me Hagas Llorar
3,1,Mastiksoul,Funk Master Flex
4,1,Detroit Grand Pubahs,Riot 66
...,...,...,...
245,1,Psapp,Apple Block
246,1,JustMe,Louder Days
247,1,Ultrabeat,The Truth Is Out There
248,1,Damero,Mope
